# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

This notebook is created to implement the Coursera assignment about Segmenting and Clustering Neighborhoods in Toronto.

We start with scraping the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe, then use geocoder to fetch the coordinates data
and merge with neighborhood data. Fianlly, we will apply the Foursquare API to explore venues for all neighborhoods in Toronto and analyze and visulize the clustering of neighborhoods.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in New York City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import warnings
warnings.filterwarnings('ignore')

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

We use the 'request' and 'BeautifulSoup' libraries to get the 'lxml' file and then find the all table tags for subsequently dataframe generation.

In [2]:
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(website_url).text

soup = BeautifulSoup(results, 'lxml')
my_table = soup.find('table', {'class': "wikitable sortable"})
table = my_table.findAll('td')

Build a loop to extract all data and store as dictionary.

In [3]:
col_list = ['PostCode', 'Borough', 'Neighborhood']
data_dict = {}
for ind, key in enumerate(col_list):
    i = ind
    value = []
    while i <= len(table)-1:
        text = table[i].text
        value.append(text)
        i += 3
    data_dict[key] = value

print('Store data in dictionary!')

Store data in dictionary!


#### Build dataframe and clean the dataset.

In [4]:
# Build and clean dataframe
data_df = pd.DataFrame(data_dict)
data_df = data_df[['PostCode', 'Borough', 'Neighborhood']]
data_df['Neighborhood'] = data_df['Neighborhood'].apply(lambda x: x.split("\n")[0])

# Drop the cells with Not assigned Borough
data_df = data_df[data_df.Borough != 'Not assigned']

# A cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
data_df['Neighborhood'][data_df.Neighborhood == 'Not assigned'] = data_df[data_df.Neighborhood == 'Not assigned']['Borough']

# Two rows are combined into one row with same PostCode and Borough.
data_df = data_df.groupby(['PostCode', 'Borough']).agg({'Neighborhood': lambda x: ' , '.join(x)}).reset_index()

In [5]:
# Have a look the data
data_df.sort_values('Neighborhood', inplace=True)
data_df = data_df.reset_index(drop=True)
data_df.head(10)

,PostCode,Borough,Neighborhood
0,M5H,Downtown Toronto,"Adelaide , King , Richmond"
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,"Agincourt North , L'Amoreaux East , Milliken ,..."
3,M9V,Etobicoke,"Albion Gardens , Beaumond Heights , Humbergate..."
4,M8W,Etobicoke,"Alderwood , Long Branch"
5,M3H,North York,"Bathurst Manor , Downsview North , Wilson Heights"
6,M2K,North York,Bayview Village
7,M5M,North York,"Bedford Park , Lawrence Manor East"
8,M5E,Downtown Toronto,Berczy Park
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [6]:
#Print the row number of the cleaned data
print('Row number of cleaned data:', data_df.shape[0])

Row number of cleaned data: 103


#### Use geocoder to fetch the coordinates data

Using geocoder to fetch the coordinates data, as it is taking long time and not reliable, the csv file is directly downloaded and used in analysis

In [7]:
# import geocoder 

# latitude = []
# longitude = []
# for post_code, borough in zip(data_df['PostCode'], data_df['Borough']):
#     print('Fetching Coordinates For:', post_code, borough)
    
#     # initialize your variable to None
#     lat_lng_coords = None

#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, {}'.format(post_code, borough))
#         lat_lng_coords = g.latlng

#     latitude.append(lat_lng_coords[0])
#     longitude.append(lat_lng_coords[1])

# data_df['Latitude'] = latitude
# data_df['Longitude'] = longitude

In [8]:
# Read coordinates CSV file
coord_df = pd.read_csv('http://cocl.us/Geospatial_data')
coord_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the coordinates into the dataset by Postcode, quickly examine the resulting dataframe.

In [9]:
#Merge the coordinates into the dataset by Postcode
coord_df = coord_df.rename(columns={'Postal Code': 'PostCode'})
data_df = data_df.merge(coord_df, on='PostCode')
data_df.head(10)

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide , King , Richmond",43.650571,-79.384568
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North , L'Amoreaux East , Milliken ,...",43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens , Beaumond Heights , Humbergate...",43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood , Long Branch",43.602414,-79.543484
5,M3H,North York,"Bathurst Manor , Downsview North , Wilson Heights",43.754328,-79.442259
6,M2K,North York,Bayview Village,43.786947,-79.385975
7,M5M,North York,"Bedford Park , Lawrence Manor East",43.733283,-79.419750
8,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


## 2. Explore the neighborhoods in Toronto.

Extractor the neighborhoods and corresponding coordiantes for Toronto. 
As this assignment is only to cluster the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [10]:
toronto_data = data_df[data_df.Borough.str.contains("Toronto")]
toronto_data.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide , King , Richmond",43.650571,-79.384568
8,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
11,M6K,West Toronto,"Brockton , Exhibition Place , Parkdale Village",43.636847,-79.428191
12,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
14,M5V,Downtown Toronto,"CN Tower , Bathurst Quay , Island airport , Ha...",43.628947,-79.394420


Since each borough might have multiple neighborhoods, therefore create function to split the neighborhoods so that each row only contains one neighborhood. 

In [11]:
def splitDataFrameList(df,target_column,separator):
    
    row_accumulator = []

    def splitListToRows(row, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s.strip()
            row_accumulator.append(new_row)
    
    df.apply(splitListToRows, axis=1, args = (separator, ))
    new_df = pd.DataFrame(row_accumulator)
    return new_df

In [12]:
toronto_data = splitDataFrameList(toronto_data, 'Neighborhood', ',')
toronto_data.head()

,Borough,Latitude,Longitude,Neighborhood,PostCode
0,Downtown Toronto,43.650571,-79.384568,Adelaide,M5H
1,Downtown Toronto,43.650571,-79.384568,King,M5H
2,Downtown Toronto,43.650571,-79.384568,Richmond,M5H
3,Downtown Toronto,43.644771,-79.373306,Berczy Park,M5E
4,West Toronto,43.636847,-79.428191,Brockton,M6K


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'I2VM1WKFQDCXYK1VC4CONME50KKKEWC1MIMOYBHNZOTFOH3F' # your Foursquare ID
CLIENT_SECRET = '2OALVQNJ4IIXJFL2C4TD0HWSZ4EDSGBEXYHV4GUYYSQSL5V3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I2VM1WKFQDCXYK1VC4CONME50KKKEWC1MIMOYBHNZOTFOH3F
CLIENT_SECRET:2OALVQNJ4IIXJFL2C4TD0HWSZ4EDSGBEXYHV4GUYYSQSL5V3


Let's create a function to get the top 100 venues that are in each neighborhoods within a radius of 500 meters in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Adelaide
King
Richmond
Berczy Park
Brockton
Exhibition Place
Parkdale Village
Business Reply Mail Processing Centre 969 Eastern
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Cabbagetown
St. James Town
Central Bay Street
Chinatown
Grange Park
Kensington Market
Christie
Church and Wellesley
Commerce Court
Victoria Hotel
Davisville
Davisville North
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
Design Exchange
Toronto Dominion Centre
Dovercourt Village
Dufferin
First Canadian Place
Underground city
Forest Hill North
Forest Hill West
Harbord
University of Toronto
Harbourfront
Regent Park
Harbourfront East
Toronto Islands
Union Station
High Park
The Junction South
Lawrence Park
Little Portugal
Trinity
Moore Park
Summerhill East
North Toronto West
Parkdale
Roncesvalles
Rosedale
Roselawn
Runnymede
Swansea
Ryerson
Garden District
St. James Town
Stn A PO Boxes 25 The Esplanade
Studio District
The Annex
North Midtown
Yorkville

Have a Look at the venues number of all neighborhood in Toronto.

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(3327, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adelaide,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650609,-79.386280,Concert Hall
1,Adelaide,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,Adelaide,43.650571,-79.384568,The Keg Steakhouse & Bar,43.649937,-79.384196,Steakhouse
3,Adelaide,43.650571,-79.384568,Estiatorio Volos,43.650329,-79.384533,Greek Restaurant
4,Adelaide,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel


Let's look at how many venues for each neighborhood

In [17]:
toronto_venues.groupby('Neighborhood').Venue.count()

Neighborhood
Adelaide                                             100
Bathurst Quay                                         15
Berczy Park                                           57
Brockton                                              22
Business Reply Mail Processing Centre 969 Eastern     19
CN Tower                                              15
Cabbagetown                                           50
Central Bay Street                                    86
Chinatown                                            100
Christie                                              16
Church and Wellesley                                  89
Commerce Court                                       100
Davisville                                            35
Davisville North                                       9
Deer Park                                             14
Design Exchange                                      100
Dovercourt Village                                    20
Dufferin          

#### Let's find out how many unique categories can be curated from all the returned venues

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


## 3. Analyze Each Neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
toronto_onehot.shape

(3327, 236)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.00,0.000000
1,Bathurst Quay,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.00,0.000000
3,Brockton,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,CN Tower,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Cabbagetown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,Central Bay Street,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.011628,0.000000,0.000000,0.011628,0.00,0.000000
8,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.01,0.010000,0.000000,0.00,0.060000,0.000000,0.050000,0.010000,0.00,0.000000
9,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


#### Let's confirm the new size

In [22]:
toronto_grouped.shape

(73, 236)

#### Let's print each neighborhood along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Bathurst Quay----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3   Harbor / Marina  0.07
4             Plane  0.07


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3              Bakery  0.04
4  Seafood Restaurant  0.04


----Brockton----
                venue  freq
0      Breakfast Spot  0.09
1                Café  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4       Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2         Pizza Place  0.05
3             Brewery  0.05
4          Smoke Shop  0.05


----CN Tower----
     

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
2,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Cheese Shop,Restaurant,Bakery,Farmers Market,Steakhouse,Café,Seafood Restaurant
3,Brockton,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Recording Studio,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data[['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'PostCode']]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Adelaide,43.650571,-79.384568,M5H,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
1,Downtown Toronto,King,43.650571,-79.384568,M5H,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
2,Downtown Toronto,Richmond,43.650571,-79.384568,M5H,0,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
3,Downtown Toronto,Berczy Park,43.644771,-79.373306,M5E,0,Coffee Shop,Cocktail Bar,Pub,Cheese Shop,Restaurant,Bakery,Farmers Market,Steakhouse,Café,Seafood Restaurant
4,West Toronto,Brockton,43.636847,-79.428191,M6K,0,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store


#### Use geopy library to get the latitude and longitude values of Toronto.

In [28]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Finally, let's visualize the resulting clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
1,King,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
2,Richmond,Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Hotel,Sushi Restaurant,Bakery,Bar,Asian Restaurant
3,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Cheese Shop,Restaurant,Bakery,Farmers Market,Steakhouse,Café,Seafood Restaurant
4,Brockton,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
5,Exhibition Place,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
6,Parkdale Village,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Pet Store
7,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Recording Studio,Smoke Shop,Brewery,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant
15,Cabbagetown,Coffee Shop,Restaurant,Pizza Place,Italian Restaurant,Café,Pub,Bakery,Park,Farmers Market,Japanese Restaurant
16,St. James Town,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Pizza Place,Park,Cosmetics Shop,Bakery,Breakfast Spot


#### Cluster 2

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Moore Park,Gym,Playground,Grocery Store,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
53,Summerhill East,Gym,Playground,Grocery Store,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
9,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
10,Island airport,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
11,Harbourfront West,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
12,King and Spadina,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
13,Railway Lands,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport
14,South Niagara,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Plane,Airport Gate,Airport


#### Cluster 4

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Forest Hill North,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
39,Forest Hill West,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
57,Rosedale,Park,Playground,Trail,Wings Joint,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 5

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Roselawn,Garden,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
